# 🚀 Automatic1111 + LoRA + DeepDanbooru + Anti-Idle
Esse notebook monta o Stable Diffusion WebUI no Colab, já integrado com Google Drive para LoRAs, DeepDanbooru para prompts automáticos e proteção contra queda por inatividade.

In [ ]:
# Monta o Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clona o repositório Automatic1111
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

# Conecta a pasta do Google Drive onde ficam os LoRAs
!mkdir -p /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/LoRA /content/stable-diffusion-webui/models/Lora

In [ ]:
# Instalações extras: DeepDanbooru
!pip install tensorflow
!git clone https://github.com/KichangKim/DeepDanbooru /content/DeepDanbooru
!pip install -r /content/DeepDanbooru/requirements.txt

In [ ]:
# Função para gerar prompt a partir de uma imagem com DeepDanbooru (threshold = 0.3)
import os
import tensorflow as tf
from PIL import Image
import numpy as np
from IPython.display import clear_output

MODEL_PATH = '/content/DeepDanbooru/deepdanbooru-v3-20211112-sgd-e28'
PROJECT_CONTEXT = tf.saved_model.load(MODEL_PATH)

def image_to_prompt(img_path, threshold=0.3):
    image = Image.open(img_path).convert('RGB').resize((512, 512))
    image_array = np.expand_dims(np.array(image, dtype=np.float32) / 255, axis=0)
    predictions = PROJECT_CONTEXT(image_array)[0].numpy()
    tags = []
    with open(os.path.join(MODEL_PATH, 'tags.txt'), 'r') as f:
        tag_list = f.read().splitlines()
    for tag, prob in zip(tag_list, predictions):
        if prob >= threshold:
            tags.append(tag.replace('_',' '))
    return ', '.join(tags)

In [ ]:
# Exemplo: gerar prompt de uma imagem
# prompt = image_to_prompt('/content/minha_imagem.jpg', threshold=0.3)
# print(prompt)

In [ ]:
# Inicia o WebUI
!COMMANDLINE_ARGS="--skip-torch-cuda-test --no-half --precision full" \
  REQS_FILE="requirements.txt" \
  python launch.py --share

In [ ]:
%%javascript
function ClickConnect(){
  console.log("✅ Colab anti-idle ativado");
  var btn = document.querySelector("colab-toolbar-button#connect"); 
  if (btn) {
    btn.click();
  }
}
setInterval(ClickConnect, 60000);  // 60 segundos